In [1]:
from keras.models import load_model

import matplotlib.pyplot as plt
import os.path
import sys
import pprint
import numpy as np

from data import DataSet

pprint.pprint(sys.path)

HOMEDIR = os.path.expanduser("~") + '/'
DATA_PATH = HOMEDIR + "dataset/UCF101/data_file.csv"
FRAME_DIR = HOMEDIR + "dataset/UCF101/UCF-101_frame"
TRAIN_DIR = FRAME_DIR + "/train"
TEST_DIR = FRAME_DIR + "/test"
MODEL_DIR = HOMEDIR + "notebook/201907/videoActionRecognitionStudy/data/checkpoints/"


Using TensorFlow backend.


['/home/s-sato/notebook/201907/videoActionRecognitionStudy',
 '/home/s-sato/anaconda3/lib/python37.zip',
 '/home/s-sato/anaconda3/lib/python3.7',
 '/home/s-sato/anaconda3/lib/python3.7/lib-dynload',
 '',
 '/home/s-sato/.local/lib/python3.7/site-packages',
 '/home/s-sato/anaconda3/lib/python3.7/site-packages',
 '/home/s-sato/notebook/library',
 '/home/s-sato/anaconda3/lib/python3.7/site-packages/IPython/extensions',
 '/home/s-sato/.ipython']


In [2]:
def predictStart(model_type="mlp",model_name="",seq_length=40,class_limit=4
                 ,video_name="v_ApplyLipstick_g01_c01"):
    
    saved_model = MODEL_DIR + model_name

    #Chose images or features and image shape based on network.
    if model_type in ['conv_3d', 'c3d', 'lrcn']:
        data_type = 'images'
        image_shape = (80, 80, 3)
    elif model_type in ['lstm', 'mlp']:
        data_type = 'features'
        image_shape = None
    else:
        raise ValueError("Invalid model. See train.py for options.")
        
    model = load_model(saved_model)
    
    # Get the data and process it.
    if image_shape is None:
        data = DataSet(seq_length=seq_length, class_limit=class_limit,data_path=DATA_PATH)
    else:
        data = DataSet(seq_length=seq_length, image_shape=image_shape,
            class_limit=class_limit,data_path=DATA_PATH)

    # Extract the sample from the data.
    sample = data.get_frames_by_filename(video_name, data_type, FRAME_DIR)

    # Predict!
    prediction = model.predict(np.expand_dims(sample, axis=0))
    
    print(model_type + " accuracy")
    #print(prediction)
    data.print_class_from_prediction(np.squeeze(prediction, axis=0))
    print("")


In [3]:
predictStart(model_type="conv_3d",model_name="conv_3d-images.006-3.226.hdf5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
conv_3d accuracy
ApplyLipstick: 0.37
ApplyEyeMakeup: 0.12
Archery: 0.00
BabyCrawling: 0.00



In [4]:
predictStart(model_type="mlp",model_name="mlp-features.001-2.023.hdf5")

mlp accuracy
ApplyEyeMakeup: 0.29
ApplyLipstick: 0.12
BabyCrawling: 0.00
Archery: 0.00



In [5]:
predictStart(model_type="lstm",model_name="lstm-features.002-3.502.hdf5")

lstm accuracy
ApplyEyeMakeup: 0.17
ApplyLipstick: 0.03
BabyCrawling: 0.01
Archery: 0.01

